In [1]:
import pandas as pd

dados_treino = pd.read_csv('/content/drive/MyDrive/curso_word2vec_treinamento/treino.csv')
dados_teste = pd.read_csv('/content/drive/MyDrive/curso_word2vec_treinamento/teste.csv')

In [2]:
dados_treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


### Spacy

In [3]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download pt_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize

In [4]:
import spacy

nlp = spacy.load('pt_core_news_sm')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
doc = nlp('Python é uma boa linguagem de programação.')

In [6]:
print(doc, type(doc))
print(doc[0], type(doc[0]))

Python é uma boa linguagem de programação. <class 'spacy.tokens.doc.Doc'>
Python <class 'spacy.tokens.token.Token'>


In [7]:
doc[0].is_stop

False

In [8]:
def tratar_textos(doc):
  tokens_validos = []

  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)

  if len(tokens_validos) > 2:
    return ' '.join(tokens_validos)

tratar_textos(doc)

'Python linguagem programação'

In [9]:
textos_para_tratamento = (titulo.lower() for titulo in dados_treino['title'])

In [10]:
# Utilize o metodo pipe para utilizar geradores
textos_tratados = [tratar_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=-1)]

In [11]:
titulos_tratados = pd.DataFrame({'titulo': textos_tratados})

In [12]:
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


### Hiperparâmetros

In [74]:
from gensim.models import Word2Vec

# sg=0 cbow, sg=1 skip-gram
# window=2 utiliza 2 palavras antes e depois para entender o contexto e treinar o modelo
# size=300 dimensão do vetor
# min_count=5 desconcidera as palavras que aparecem menos de 5 vezes
# alpha=0.03 taxa de aprendizagem
# min_alpha decaimento do valor de alpha
w2v_modelo = Word2Vec(
    sg=0,
    window=2,
    size=300,
    min_count=5,
    alpha=0.03,
    min_alpha=0.007,
)

In [101]:
# tratando os dados para vocabulario e treinamento
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
matriz_vocab = [titulo.split(' ') for titulo in titulos_tratados.titulo]

In [77]:
# Constroi o Vocabulario
# Recebe um matriz com os tokens
# progress_per mostrar os logs a cada 5000 iterações
w2v_modelo.build_vocab(matriz_vocab, progress_per=5000)

In [78]:
# Treinamento
w2v_modelo.train(matriz_vocab, total_examples=w2v_modelo.corpus_count, epochs=30)

(14583895, 16207260)

In [100]:
w2v_modelo.wv.most_similar('marvel')

[('mr', 0.6899381875991821),
 ('teaser', 0.6877329349517822),
 ('sensualidade', 0.681988000869751),
 ('robot', 0.6767659187316895),
 ('adaptações', 0.6740226745605469),
 ('spielberg', 0.6667453050613403),
 ('mary', 0.6614916324615479),
 ('estranho', 0.6513721942901611),
 ('hqs', 0.649604856967926),
 ('sofia', 0.6493239402770996)]

#### Treinando com skip-gram

In [102]:
# Skipgram funciona um pouco melhor com windows maiores
w2v_modelo_sg = Word2Vec(
    sg=1,
    window=5,
    size=300,
    min_count=5,
    alpha=0.03,
    min_alpha=0.007,
)

# Criando vocabulario
w2v_modelo_sg.build_vocab(matriz_vocab, progress_per=5000)

# Treinando modelo
w2v_modelo_sg.train(matriz_vocab, total_examples=w2v_modelo_sg.corpus_count, epochs=30)

(14585502, 16207260)

In [103]:
w2v_modelo_sg.wv.most_similar('marvel')

[('desventuras', 0.5488990545272827),
 ('tina', 0.5422924757003784),
 ('galáxia', 0.5395383238792419),
 ('toy', 0.5384949445724487),
 ('demolidor', 0.5280839800834656),
 ('cage', 0.5132519006729126),
 ('sofia', 0.5061680674552917),
 ('robot', 0.5046441555023193),
 ('fey', 0.5030457973480225),
 ('theory', 0.5010255575180054)]

### Salvando o modelos

In [104]:
w2v_modelo.wv.save_word2vec_format('/content/drive/MyDrive/curso_word2vec_treinamento/modelo_cbow.txt', binary=False)
w2v_modelo_sg.wv.save_word2vec_format('/content/drive/MyDrive/curso_word2vec_treinamento/modelo_sg.txt', binary=False)